## PROJET GLO-4027
Prédiction éléction

## Lecture des données

In [89]:
import numpy as np
import pandas as pd

In [90]:
df=pd.read_csv("./../CES19.csv",parse_dates=['cps19_StartDate'])
dfTest=df[:2000]

C:\Users\Jean\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (25,27,28,30,31,33,34,39,179,181,188,192,193,199,210,227,234,236,237,239,240,243,276,299,302,311,314,315,316,327,362,417,418,421,422,423,424,425,431,464,465,468,538,541,578,579,582,583,584) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
#Pour tester les algo avec moins de données
df=dfTest
df.isnull().sum()

Unnamed: 0                            0
cps19_StartDate                       0
cps19_EndDate                         0
cps19_ResponseId                      0
cps19_consent                         0
                                   ... 
pes19_inattentive                  1156
cps19_weight_general_all              4
cps19_weight_general_restricted     314
pes19_weight_general_all           1157
pes19_weight_general_restricted    1220
Length: 621, dtype: int64

In [91]:
attributs=np.array(df.columns)

In [92]:
dfTrain=df[ (pd.isna(df['cps19_votechoice'])==False) | (pd.isna(df['cps19_votechoice_pr'])==False)
           |(pd.isna(df['cps19_vote_unlikely'])==False) | (pd.isna(df['cps19_vote_unlike_pr'])==False)
           |(pd.isna(df['cps19_v_advance'])==False)]
dfPredire=df[ (pd.isna(df['cps19_votechoice'])==True) & (pd.isna(df['cps19_votechoice_pr'])==True)
           &(pd.isna(df['cps19_vote_unlikely'])==True) & (pd.isna(df['cps19_vote_unlike_pr'])==True)
           &(pd.isna(df['cps19_v_advance'])==True)]
len(dfTrain)/len(df)
len(dfPredire)

4885

## ANALYSE statistique des données

% de data par attributs

In [5]:
def pourcentage_par_attributs(dataframe,attribut):
    maskNaN=np.array(dataframe[attribut].isna())
    compteur = np.count_nonzero(maskNaN == False)
    return compteur/len(maskNaN)

In [6]:
pourcentage_response=[]
for attribut in attributs :
    pourcentage_response.append([attribut,pourcentage_par_attributs(dfTrain,attribut)])
pourcentage_response

[['Unnamed: 0', 1.0],
 ['cps19_StartDate', 1.0],
 ['cps19_EndDate', 1.0],
 ['cps19_ResponseId', 1.0],
 ['cps19_consent', 1.0],
 ['cps19_citizenship', 1.0],
 ['cps19_yob', 1.0],
 ['cps19_yob_2001_age', 0.007468804080517351],
 ['cps19_gender', 1.0],
 ['cps19_province', 1.0],
 ['cps19_education', 1.0],
 ['cps19_demsat', 1.0],
 ['cps19_imp_iss', 0.8107599356346965],
 ['cps19_imp_iss_party', 0.8111546285332605],
 ['cps19_imp_iss_party_7_TEXT', 0.014785803200048577],
 ['cps19_imp_loc_iss', 0.35920089868536903],
 ['cps19_imp_loc_iss_p', 0.3593830646385524],
 ['cps19_imp_loc_iss_p_7_TEXT', 0.006649057291192276],
 ['cps19_interest_gen_1', 0.9372438291283359],
 ['cps19_interest_elxn_1', 0.9409478701763974],
 ['cps19_v_likely', 0.9646294440902329],
 ['cps19_v_likely_pr', 0.035370555909767135],
 ['cps19_votechoice', 0.8628897592373319],
 ['cps19_votechoice_7_TEXT', 0.005495339587697726],
 ['cps19_votechoice_pr', 0.03518838995658378],
 ['cps19_votechoice_pr_7_TEXT', 9.108297659167502e-05],
 ['cps19

In [7]:
#Moyenne du pourcentage de response par attribut
mean=0
for attr in pourcentage_response :
    mean+=attr[1]
mean/len(pourcentage_response)

0.31378339666362365

Fusionner attribut :

In [8]:
def fusion_attribut_vote(attributs_fusion,attribut_name):
    #priorité au premiers attributs de attributs_fusion
    erase=0
    tab=np.array(dfTrain[attributs_fusion])
    maskNaN=np.array(dfTrain[attributs_fusion].isna())
    new_tab=[]
    for i in range(len(tab)):
        new_tab.append(np.nan)
        if np.count_nonzero(maskNaN[i]==False)>=2 :
            if tab[i][3]!=tab[i][6] :
                if tab[i][3]=="Don't know/ Prefer not to answer":
                    maskNaN[i][3]=True
                else :
                    erase+=np.count_nonzero(maskNaN[i]==False)-1
                    print(tab[i])
        for j in range(len(tab[i])):
            if not maskNaN[i][j] :
                new_tab[-1]=tab[i][j]
                continue
    dfNan = dfTrain.assign(vote_choice = new_tab)
    print(erase)
    return dfNan

In [9]:
dfTrain=fusion_attribut_vote(['cps19_votechoice','cps19_votechoice_pr','cps19_vote_unlikely','cps19_vote_unlike_pr','cps19_v_advance'],'vote_choice')
dfTrain['vote_choice']

0


0                             Green Party
1        Don't know/ Prefer not to answer
2                           Liberal Party
3                      Conservative Party
4        Don't know/ Prefer not to answer
                       ...               
37816                                 ndp
37817                       Liberal Party
37818                         Green Party
37820                                 ndp
37821    Don't know/ Prefer not to answer
Name: vote_choice, Length: 32937, dtype: object

In [10]:
pourcentage_par_attributs(dfTrain,'vote_choice')

1.0

Séparation par parti

In [11]:
dfAnother=dfTrain[dfTrain['vote_choice']=='Another party (please specify)']
dfBlocQuebecois=dfTrain[dfTrain['vote_choice']=='Bloc Qu<e9>b<e9>cois']# Bloc Québécois
dfConservative=dfTrain[dfTrain['vote_choice']=='Conservative Party']# • Conservative Party
dfNo=dfTrain[dfTrain['vote_choice']=="Don't know/ Prefer not to answer"]# • Don't know/ Prefer not to answer
dfGreen=dfTrain[dfTrain['vote_choice']=='Green Party']# • Green Party
dfLiberal=dfTrain[dfTrain['vote_choice']=='Liberal Party']# • Liberal Party
dfNdp=dfTrain[dfTrain['vote_choice']=='ndp']# • NDP
dfPeople=dfTrain[dfTrain['vote_choice']=="People's Party"]# • People's Party

(len(dfAnother)+len(dfBlocQuebecois)+len(dfConservative)+len(dfNo)+len(dfGreen)+len(dfLiberal)+len(dfNdp)+len(dfPeople))/len(dfTrain)

1.0

In [12]:
def coherence_attr(dfVote,attribut,confiance):
    val_attr={"nan":0}
    val_rep={}
    
    for elmt in dfVote[attribut]:
        if elmt in val_attr :
            val_attr[elmt]+=1
        if pd.isna(elmt):
            val_attr["nan"]+=1
        else :
            val_attr[elmt]=1
    for val in val_attr :
        size=len(dfTrain[dfTrain[attribut]==val])
        if size > 5 :
            if val_attr[val]/size >= confiance :
                val_rep[val]= val_attr[val]/size
    return val_rep

In [13]:
def coherence_parti(dfVote,confiance):
    confiance_parti={}
    for attr in dfVote.columns :
        if attr not in ['cps19_votechoice','cps19_votechoice_pr','cps19_vote_unlikely','cps19_vote_unlike_pr','cps19_v_advance']:
            dico=coherence_attr(dfVote,attr,confiance)
            if dico!={} :
                confiance_parti[attr]=dico
        
    return confiance_parti

In [14]:
coherence_parti(dfNdp,0.15)

KeyboardInterrupt: 

Age

In [15]:
compte = [0,0,0,0]
for elmt in dfTrain['cps19_yob']:
    if elmt <= 1959 :
        compte[3]+=1
    elif 1959 < elmt <= 1979 :
        compte[2]+=1
    elif 1979 < elmt <= 1994 :
        compte[1]+=1
    elif 1994 < elmt :
        compte[0]+=1

compte

[2456, 8733, 11495, 10253]

In [16]:
# cps19_yob_norm{ np.nan : 0 ; 1994< : 0.25 ; 1979<.<=1994 : 0.5 ; 1959<.<=1979 : 0.75 ; .<=1959:1 ;
norm_val_yob=[]
for elmt in dfTrain['cps19_yob']:
    if elmt <= 1959 :
        norm_val_yob.append(1)
    elif 1959 < elmt <= 1979 :
        norm_val_yob.append(0.75)
    elif 1979 < elmt <= 1994 :
        norm_val_yob.append(0.5)
    elif 1994 < elmt :
        norm_val_yob.append(0.25)
    else :
        norm_val_yob.append(0)
len(norm_val_yob)

32937

In [17]:
def analyse_attr(df,attr):
    valeurs={}
    for elmt in df[attr]:
        if elmt not in valeurs :
            valeurs[elmt]=0
        else :
            valeurs[elmt]+=1
    return valeurs

In [18]:
rep={}
for i in range(11,20):
    rep[attributs[i]]=analyse_attr(dfTrain,attributs[i])
rep

{'cps19_demsat': {'Fairly satisfied': 18599,
  "Don't know/ Prefer not to answer": 698,
  'Very satisfied': 5085,
  'Not very satisfied': 6948,
  'Not at all satisfied': 1602},
 'cps19_imp_iss': {'environnement': 208,
  nan: 6232,
  'strengthening foreign relations': 0,
  'Taxation and Government Spending': 0,
  'Immigration Issues': 0,
  'Climate Change': 244,
  'Minimum Wage': 0,
  'Climate change/environmental issues': 1,
  'climate related issues': 0,
  'economic opportunities to young people, specifically reducing barriers to owning property.': 0,
  'Making sure youth voices are heard and our problems are assessed': 0,
  'Carbon Tax': 10,
  'ABORTION RIGHTS': 0,
  'immigration': 138,
  'Education Funding and Carbon tax': 0,
  'cost of living': 101,
  'climate change': 327,
  'Issues on  the environment': 0,
  'Education; Student Funding': 0,
  'Economy': 843,
  'Environmental issues/climate change': 2,
  'education and environment': 1,
  'Climate change': 816,
  'Osap funding': 0,

In [19]:
dfGlobal=[dfAnother,dfBlocQuebecois,dfConservative,dfNo,dfGreen,dfLiberal,dfNdp,dfPeople]
partyName=["Another","BlocQuebecois","Conservative","No","Green","Liberal","Ndp","People"]

In [20]:
def importance_valeur(attr):
    pourcentage={"Another":0 ,
                 "BlocQuebecois":0,
                 "Conservative":0,
                 "No":0,
                 "Green":0,
                 "Liberal":0,
                 "Ndp":0,
                 "People":0}
    valeurs=[]
    for i in range(len(dfGlobal)) :
        pourcentage[partyName[i]]=analyse_attr(dfGlobal[i],attr)
        for val in pourcentage[partyName[i]].keys() :
            if val not in valeurs :
                valeurs.append(val)
    return pourcentage,valeurs

In [21]:
attribut='cps19_demsat'
pourcentage,valeurs=importance_valeur(attribut)
rep=[["Valeur"]]
for party in partyName :
    rep[0].append(party)
rep[0].append("Total")

for val in valeurs :
    decompte=[val+" :"]
    lTotal=0
    for party in partyName :
        try :
            decompte.append(pourcentage[party][val])
            lTotal+=pourcentage[party][val]
        except :
            decompte.append(decompte.append(0))
    decompte.append(lTotal)
    rep.append(decompte)
rep

[['Valeur',
  'Another',
  'BlocQuebecois',
  'Conservative',
  'No',
  'Green',
  'Liberal',
  'Ndp',
  'People',
  'Total'],
 ['Fairly satisfied :', 98, 838, 4759, 2850, 1311, 5868, 2622, 246, 18592],
 ['Not very satisfied :', 71, 363, 2237, 1313, 729, 840, 1161, 227, 6941],
 ['Very satisfied :', 11, 195, 1072, 497, 304, 2480, 437, 82, 5078],
 ["Don't know/ Prefer not to answer :", 4, 21, 163, 219, 84, 95, 92, 13, 691],
 ['Not at all satisfied :', 28, 75, 639, 257, 178, 86, 242, 90, 1595]]

cps19_imp_iss 18589/32937  ['cps19_imp_iss_party', 0.8111546285332605]

In [22]:
dfTrain[dfTrain['cps19_imp_iss_party']==dfTrain['vote_choice']][['cps19_imp_iss_party','vote_choice']]

,cps19_imp_iss_party,vote_choice
0,Green Party,Green Party
3,Conservative Party,Conservative Party
4,Don't know/ Prefer not to answer,Don't know/ Prefer not to answer
9,Liberal Party,Liberal Party
14,Liberal Party,Liberal Party
...,...,...
37811,Bloc Qu<e9>b<e9>cois,Bloc Qu<e9>b<e9>cois
37815,Bloc Qu<e9>b<e9>cois,Bloc Qu<e9>b<e9>cois
37817,Liberal Party,Liberal Party
37818,Green Party,Green Party


cps19_imp_loc_iss_p 7752 ['cps19_imp_loc_iss_p', 0.3593830646385524],

In [23]:
dfTrain[dfTrain['cps19_imp_loc_iss_p']==dfTrain['vote_choice']][['cps19_imp_loc_iss_p','vote_choice']]

,cps19_imp_loc_iss_p,vote_choice
10,ndp,ndp
19,ndp,ndp
27,Liberal Party,Liberal Party
32,Conservative Party,Conservative Party
42,Liberal Party,Liberal Party
...,...,...
37777,ndp,ndp
37805,Bloc Qu<e9>b<e9>cois,Bloc Qu<e9>b<e9>cois
37806,Liberal Party,Liberal Party
37811,Bloc Qu<e9>b<e9>cois,Bloc Qu<e9>b<e9>cois


 ['cps19_interest_gen_1', 0.9372438291283359],
 ['cps19_interest_elxn_1', 0.9409478701763974],

In [24]:
dfTrain['cps19_fed_gov_sat']

0        Not very satisfied
1        Not very satisfied
2          Fairly satisfied
3        Not very satisfied
4        Not very satisfied
                ...        
37816    Not very satisfied
37817      Fairly satisfied
37818    Not very satisfied
37820      Fairly satisfied
37821      Fairly satisfied
Name: cps19_fed_gov_sat, Length: 32937, dtype: object

In [48]:
def gatherPartyAttribute(attribut,attributName,newAttrName):
    newAttrTab=[]
    for index, row in dfTrain[attr].iterrows() :
        newAttrTab.append(attributName[np.argmax(row)])
    
    dfTrain[newAttrName]=newAttrTab
    
    newAttrTab=[]
    for index, row in dfPredire[attr].iterrows() :
        newAttrTab.append(attributName[np.argmax(row)])
    
    dfPredire[newAttrName]=newAttrTab
    return dfTrain[dfTrain[newAttrName]==dfTrain['vote_choice']][[newAttrName,'vote_choice']].shape

In [59]:
#'cps19_party_rating'
attr=['cps19_party_rating_23', 'cps19_party_rating_24', 'cps19_party_rating_25', 'cps19_party_rating_26', 'cps19_party_rating_27', 'cps19_party_rating_28']
attrNameParty= ['Liberal Party','Conservative Party','ndp','Bloc Québécois','Green Party',"People's Party"]

gatherPartyAttribute(attr,attrNameParty,'cps19_party_rating_max')

C:\Users\Jean\AppData\Local\Temp/ipykernel_7000/2886697018.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfPredire[newAttrName]=newAttrTab


(21730, 2)

In [50]:
#'cps19_lead_rating'
attr=['cps19_lead_rating_23', 'cps19_lead_rating_24', 'cps19_lead_rating_25', 'cps19_lead_rating_26', 'cps19_lead_rating_27', 'cps19_lead_rating_28']
attrNamePeople=['Justin Trudeau','Andrew Scheer','Jagmeet Singh','Yves-François Blanchet','Elizabeth May','Maxime Bernier']

gatherPartyAttribute(attr,attrNamePeople,'cps19_lead_rating_max')

C:\Users\Jean\AppData\Local\Temp/ipykernel_7000/2886697018.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfPredire[newAttrName]=newAttrTab


(0, 2)

In [51]:
#'cps19_cand_rating'
attr=['cps19_cand_rating_23', 'cps19_cand_rating_24', 'cps19_cand_rating_25', 'cps19_cand_rating_26', 'cps19_cand_rating_27', 'cps19_cand_rating_28']
attrNameParty= ['Liberal Party','Conservative Party','ndp','Bloc Québécois','Green Party',"People's Party"]

gatherPartyAttribute(attr,attrNameParty,'cps19_cand_rating_max')

C:\Users\Jean\AppData\Local\Temp/ipykernel_7000/2886697018.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfPredire[newAttrName]=newAttrTab


(17173, 2)

In [62]:
def gatherPreferencePeople(attribut,attributName,newAttrName):
    newAttrTab=[]
    for index, row in dfTrain[attr].iterrows() :
        uniqueValue=True
        i=0
        while i < len(row) :
            if str(row[i]) != 'nan' :
                if uniqueValue :
                    newAttrTab.append(attributName[i])
                    uniqueValue=False
                else :
                    newAttrTab[-1]=np.nan
                    i=len(row)
                    
            i+=1
        if uniqueValue :   
            newAttrTab.append(np.nan)
    
    dfTrain[newAttrName]=newAttrTab
    
    newAttrTab=[]
    for index, row in dfPredire[attr].iterrows() :
        uniqueValue=True
        i=0
        while i < len(row) :
            if str(row[i]) != 'nan' :
                if uniqueValue :
                    newAttrTab.append(attributName[i])
                    uniqueValue=False
                else :
                    newAttrTab[-1]=np.nan
                    i=len(row)
                    
            i+=1
        if uniqueValue :   
            newAttrTab.append(np.nan)
    
    dfPredire[newAttrName]=newAttrTab
    return dfTrain[dfTrain[newAttrName]==dfTrain['vote_choice']][[newAttrName,'vote_choice']].shape

In [65]:
#'cps19_lead_strong'
attr= ['cps19_lead_strong_113','cps19_lead_strong_114','cps19_lead_strong_115','cps19_lead_strong_116','cps19_lead_strong_117','cps19_lead_strong_118','cps19_lead_strong_119','cps19_lead_strong_120']
attrNamePeople=['Justin Trudeau','Andrew Scheer','Jagmeet Singh','Yves-François Blanchet','Elizabeth May','Maxime Bernier','None of these',"Don't know/ Prefer not to answer"]
attrNameParty= ['Liberal Party','Conservative Party','ndp','Bloc Québécois','Green Party',"People's Party","Another",np.nan]

gatherPreferencePeople(attr,attrNameParty,'cps19_lead_strong_max')

C:\Users\Jean\AppData\Local\Temp/ipykernel_7000/1562130080.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfPredire[newAttrName]=newAttrTab


(8349, 2)

In [66]:
#'cps19_lead_trust'
attr= ['cps19_lead_trust_113','cps19_lead_trust_114','cps19_lead_trust_115','cps19_lead_trust_116','cps19_lead_trust_117','cps19_lead_trust_118','cps19_lead_trust_119','cps19_lead_trust_120']
attrNamePeople=['Justin Trudeau','Andrew Scheer','Jagmeet Singh','Yves-François Blanchet','Elizabeth May','Maxime Bernier','None of these',"Don't know/ Prefer not to answer"]
attrNameParty= ['Liberal Party','Conservative Party','ndp','Bloc Québécois','Green Party',"People's Party","Another",np.nan]

gatherPreferencePeople(attr,attrNameParty,'cps19_lead_trust_max')

C:\Users\Jean\AppData\Local\Temp/ipykernel_7000/1562130080.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfPredire[newAttrName]=newAttrTab


(6966, 2)

In [67]:
#'cps19_lead_cares'
attr= ['cps19_lead_cares_113','cps19_lead_cares_114','cps19_lead_cares_115','cps19_lead_cares_116','cps19_lead_cares_117','cps19_lead_cares_118','cps19_lead_cares_119','cps19_lead_cares_120']
attrNamePeople=['Justin Trudeau','Andrew Scheer','Jagmeet Singh','Yves-François Blanchet','Elizabeth May','Maxime Bernier','None of these',"Don't know/ Prefer not to answer"]
attrNameParty= ['Liberal Party','Conservative Party','ndp','Bloc Québécois','Green Party',"People's Party","Another",np.nan]

gatherPreferencePeople(attr,attrNameParty,'cps19_lead_cares_max')

C:\Users\Jean\AppData\Local\Temp/ipykernel_7000/1562130080.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfPredire[newAttrName]=newAttrTab


(7619, 2)

## Normalisation

In [68]:
# Normalisation sur [0,1] sur des valeurs sans aucun ordre, la valeur np.nan est toujours présente et égale à 0
def normaliseNaif(dfInit,dfNorm,attribut,log):
    valeurs={ np.nan : 0 }
    for elmt in dfInit[attribut]:
        if str(elmt) not in valeurs :
            valeurs[str(elmt)]=0
    nValeur=len(valeurs)-1
    i = 0
    for val in valeurs.keys():
        valeurs[val]=i/nValeur
        i+=1
    
    norm_val=[]
    for elmt in dfInit[attribut]:
        norm_val.append(valeurs[str(elmt)])
        
    char=attribut+"_norm"
    dfNorm[char]=norm_val
    
    if log : 
        print(valeurs)
        
    return valeurs

## ENTRAINEMENT

Données d'entrainement :

In [69]:
attr_selectionne=['cps19_imp_iss_party','cps19_imp_loc_iss_p','cps19_province','cps19_education','pes19_rural_urban','cps19_demsat','cps19_gender','cps19_citizenship']

attr_selectionne=['cps19_issue_handle_1','cps19_issue_handle_2','cps19_issue_handle_3','cps19_issue_handle_4','cps19_issue_handle_5','cps19_issue_handle_6','cps19_issue_handle_7','cps19_issue_handle_8','cps19_lead_cares_max','cps19_lead_strong_max','cps19_party_rating_max','cps19_lead_rating_max','cps19_cand_rating_max','cps19_imp_iss_party','cps19_fed_gov_sat','cps19_imp_loc_iss_p']#75.65270188221008 %
#attr_selectionne=[]

dfX=dfTrain[attr_selectionne]
dfY=dfTrain['vote_choice']
dfX

,cps19_issue_handle_1,cps19_issue_handle_2,cps19_issue_handle_3,cps19_issue_handle_4,cps19_issue_handle_5,cps19_issue_handle_6,cps19_issue_handle_7,cps19_issue_handle_8,cps19_lead_cares_max,cps19_lead_strong_max,cps19_party_rating_max,cps19_lead_rating_max,cps19_cand_rating_max,cps19_imp_iss_party,cps19_fed_gov_sat,cps19_imp_loc_iss_p
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Green Party,Elizabeth May,Green Party,Green Party,Not very satisfied,ndp
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Green Party,NaN,Green Party,Elizabeth May,Green Party,Green Party,Not very satisfied,ndp
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ndp,NaN,Liberal Party,Justin Trudeau,People's Party,NaN,Fairly satisfied,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Conservative Party,Conservative Party,Andrew Scheer,Conservative Party,Conservative Party,Not very satisfied,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ndp,NaN,Conservative Party,Andrew Scheer,People's Party,Don't know/ Prefer not to answer,Not very satisfied,Conservative Party
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37816,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ndp,Jagmeet Singh,People's Party,Green Party,Not very satisfied,Don't know/ Prefer not to answer
37817,ndp,Liberal Party,Green Party,ndp,Liberal Party,Liberal Party,Liberal Party,Liberal Party,NaN,NaN,Liberal Party,Justin Trudeau,Liberal Party,Liberal Party,Fairly satisfied,NaN
37818,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Green Party,Green Party,Green Party,Elizabeth May,Green Party,Green Party,Not very satisfied,NaN
37820,ndp,ndp,Green Party,ndp,Liberal Party,Liberal Party,Liberal Party,Conservative Party,ndp,ndp,ndp,Jagmeet Singh,Liberal Party,ndp,Fairly satisfied,NaN


Randomn Forest
(https://www.stat4decision.com/fr/foret-aleatoire-avec-python/)

In [70]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split


Normalisation pour random Forest

In [71]:
dfNorm= pd.DataFrame()
for attr in attr_selectionne :
    try :
        normaliseNaif(dfX,dfNorm,attr,False)
    except:
        print(attr)

    
#dfNorm['cps19_yob_norm']=norm_val_yob

In [72]:
dfNorm

,cps19_issue_handle_1_norm,cps19_issue_handle_2_norm,cps19_issue_handle_3_norm,cps19_issue_handle_4_norm,cps19_issue_handle_5_norm,cps19_issue_handle_6_norm,cps19_issue_handle_7_norm,cps19_issue_handle_8_norm,cps19_lead_cares_max_norm,cps19_lead_strong_max_norm,cps19_party_rating_max_norm,cps19_lead_rating_max_norm,cps19_cand_rating_max_norm,cps19_imp_iss_party_norm,cps19_fed_gov_sat_norm,cps19_imp_loc_iss_p_norm
0,0.125,0.125,0.125,0.125,0.125,0.125,0.125,0.125,0.125,0.125,0.166667,0.166667,0.166667,0.111111,0.2,0.111111
1,0.125,0.125,0.125,0.125,0.125,0.125,0.125,0.125,0.250,0.125,0.166667,0.166667,0.166667,0.111111,0.2,0.111111
2,0.125,0.125,0.125,0.125,0.125,0.125,0.125,0.125,0.375,0.125,0.333333,0.333333,0.333333,0.222222,0.4,0.222222
3,0.125,0.125,0.125,0.125,0.125,0.125,0.125,0.125,0.125,0.250,0.500000,0.500000,0.500000,0.333333,0.2,0.222222
4,0.125,0.125,0.125,0.125,0.125,0.125,0.125,0.125,0.375,0.125,0.500000,0.500000,0.333333,0.444444,0.2,0.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32932,0.125,0.125,0.125,0.125,0.125,0.125,0.125,0.125,0.125,0.125,0.666667,0.666667,0.333333,0.111111,0.2,0.555556
32933,0.375,0.250,0.250,0.250,0.500,0.500,0.375,0.250,0.125,0.125,0.333333,0.333333,0.666667,0.555556,0.4,0.222222
32934,0.125,0.125,0.125,0.125,0.125,0.125,0.125,0.125,0.250,0.750,0.166667,0.166667,0.166667,0.111111,0.2,0.222222
32935,0.375,0.375,0.250,0.250,0.500,0.500,0.375,0.375,0.375,0.375,0.666667,0.666667,0.666667,0.666667,0.4,0.222222


In [73]:
x_train, x_test, y_train, y_test  = train_test_split(dfNorm, 
                                                     dfY,
                                                     test_size=0.20, 
                                                     random_state=42)

In [74]:
modele_rf = RandomForestClassifier(
     n_estimators=100,
     criterion='gini',
     max_depth=None,
     min_samples_split=50,
     min_samples_leaf=25,
     min_weight_fraction_leaf=0.0,
     max_features='auto',
     max_leaf_nodes=None,
     min_impurity_decrease=0.0,
     bootstrap=False,
     oob_score=False,
     n_jobs=None,
     random_state=None,
     verbose=0,
     warm_start=False,
     class_weight=None,
     ccp_alpha=0.0,
     max_samples=None,)

In [75]:
modele_rf.fit(x_train, y_train)

RandomForestClassifier(bootstrap=False, min_samples_leaf=25,
                       min_samples_split=50)

In [76]:
from sklearn.metrics import accuracy_score, confusion_matrix
print(f"Le pourcentage de bien classés est de : {accuracy_score(y_test, modele_rf.predict(x_test))*100} %")

Le pourcentage de bien classés est de : 75.80449301760778 %


Max : attr_selectionne=attr_selectionne=attr_selectionne=['cps19_lead_cares_max','cps19_lead_strong_max','cps19_party_rating_max','cps19_lead_rating_max','cps19_cand_rating_max','cps19_imp_iss_party','cps19_fed_gov_sat','cps19_imp_loc_iss_p']#75.65270188221008 %

Analyse random Forest

In [77]:
pd.DataFrame(modele_rf.feature_importances_,
              index = x_train.columns, 
              columns = ["importance"]).sort_values(
     "importance", 
     ascending = False)

,importance
cps19_party_rating_max_norm,0.348531
cps19_imp_iss_party_norm,0.178100
cps19_lead_rating_max_norm,0.171607
cps19_cand_rating_max_norm,0.135001
cps19_fed_gov_sat_norm,0.034989
cps19_issue_handle_6_norm,0.020640
cps19_issue_handle_8_norm,0.020403
cps19_lead_cares_max_norm,0.020013
cps19_issue_handle_2_norm,0.013214
cps19_issue_handle_1_norm,0.012232


Naif Bayes

In [78]:
from sklearn.naive_bayes import BernoulliNB
# initializaing the NB
classifer = BernoulliNB()
from sklearn.naive_bayes import GaussianNB

classifer1=GaussianNB()

# training the model
classifer.fit(x_train, y_train)
classifer1.fit(x_train, y_train)

# testing the model
y_pred = classifer.predict(x_test)
y_pred1 = classifer1.predict(x_test)

In [79]:
print("Bernoulli : ",accuracy_score(y_pred, y_test))
print("Gaussian : ",accuracy_score(y_pred1, y_test))

Bernoulli :  0.28718882817243474
Gaussian :  0.637370977534912


## Application

In [83]:
dfPredireNorm= pd.DataFrame()
for attr in attr_selectionne :
    try :
        normaliseNaif(dfPredire,dfPredireNorm,attr,False)
    except:
        print(attr)

In [102]:
dfPredire['vote_choice']=modele_rf.predict(dfPredireNorm)
dfPredire[['Unnamed: 0','vote_choice']]


C:\Users\Jean\AppData\Local\Temp/ipykernel_7000/4189917358.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfPredire['vote_choice']=modele_rf.predict(dfPredireNorm)


,Unnamed: 0,vote_choice
7,8,Green Party
22,23,Liberal Party
45,46,Conservative Party
59,60,Liberal Party
63,64,Conservative Party
...,...,...
37791,37792,Don't know/ Prefer not to answer
37792,37793,Bloc Qu<e9>b<e9>cois
37793,37794,Liberal Party
37797,37798,Conservative Party


In [136]:
f=open('prediction.txt','w')
for pred in dfPredire[['Unnamed: 0','vote_choice']].iterrows():
    string=str(pred[1][0])+" "+pred[1][1]+"\n"
    #print(string)
    f.write(string)
f.close()

In [145]:
tab=[]
ftest=open('exemple.txt','r')
for line in ftest.readlines():
    #print(line.split())
    tab.append(int(line.split()[0]))
ftest.close()

In [154]:
tab=np.sort(tab)
f=open('prediction_restricted.txt','w')
compteur=0
for pred in dfPredire[['Unnamed: 0','vote_choice']].iterrows():
    print(pred[1][0]," : ",tab[compteur])
    if(pred[1][0]-1 in tab):
        string=str(pred[1][0])+" "+pred[1][1]+"\n"
        #print(string)
        f.write(string)
        compteur+=1
f.close()
compteur

8  :  7
23  :  7
46  :  7
60  :  7
64  :  7
78  :  7
92  :  7
94  :  7
133  :  7
140  :  7
146  :  7
165  :  7
169  :  7
171  :  7
176  :  7
182  :  7
183  :  22
184  :  45
191  :  45
194  :  45
209  :  45
215  :  45
220  :  45
223  :  45
231  :  45
234  :  45
236  :  45
247  :  45
261  :  45
264  :  45
267  :  45
271  :  45
285  :  45
291  :  45
296  :  45
299  :  45
302  :  45
309  :  45
329  :  45
330  :  59
339  :  59
347  :  59
355  :  59
357  :  59
362  :  59
366  :  59
368  :  59
370  :  59
395  :  59
396  :  63
403  :  63
405  :  63
413  :  63
431  :  63
447  :  63
467  :  63
469  :  63
475  :  63
480  :  63
485  :  63
494  :  63
499  :  63
502  :  63
510  :  63
520  :  63
526  :  63
527  :  77
534  :  77
547  :  77
556  :  77
557  :  91
569  :  91
579  :  91
586  :  91
602  :  91
619  :  91
631  :  91
635  :  91
636  :  93
641  :  93
643  :  93
644  :  132
655  :  132
668  :  132
676  :  132
686  :  132
687  :  132
695  :  132
714  :  132
722  :  132
725  :  132
743  :  132
74

6600  :  785
6613  :  785
6618  :  785
6624  :  785
6628  :  785
6630  :  785
6638  :  785
6640  :  785
6641  :  785
6642  :  785
6645  :  785
6649  :  785
6650  :  789
6654  :  789
6691  :  789
6711  :  789
6715  :  789
6734  :  789
6735  :  793
6746  :  793
6763  :  793
6789  :  793
6791  :  793
6795  :  793
6808  :  793
6809  :  793
6816  :  793
6820  :  793
6827  :  793
6831  :  793
6859  :  793
6868  :  793
6881  :  793
6890  :  793
6897  :  793
6898  :  801
6901  :  801
6908  :  801
6922  :  801
6924  :  801
6926  :  801
6935  :  801
6942  :  801
6945  :  801
6954  :  801
6966  :  801
6973  :  801
6983  :  801
6984  :  804
6996  :  804
7004  :  804
7006  :  804
7008  :  804
7018  :  804
7027  :  804
7033  :  804
7043  :  804
7058  :  804
7064  :  804
7068  :  804
7070  :  804
7081  :  804
7082  :  807
7107  :  807
7120  :  807
7130  :  807
7138  :  807
7161  :  807
7169  :  807
7185  :  807
7194  :  807
7200  :  807
7201  :  821
7244  :  821
7254  :  821
7270  :  821
7272  :  821

13325  :  1695
13327  :  1695
13356  :  1695
13361  :  1695
13362  :  1714
13369  :  1714
13376  :  1714
13384  :  1714
13389  :  1714
13393  :  1714
13418  :  1714
13422  :  1714
13430  :  1714
13436  :  1714
13437  :  1731
13439  :  1731
13453  :  1731
13455  :  1731
13475  :  1731
13521  :  1731
13526  :  1731
13528  :  1731
13529  :  1731
13536  :  1731
13539  :  1731
13546  :  1731
13558  :  1731
13560  :  1731
13567  :  1731
13575  :  1731
13579  :  1731
13585  :  1731
13589  :  1731
13594  :  1731
13596  :  1731
13611  :  1731
13612  :  1731
13617  :  1731
13621  :  1731
13624  :  1731
13625  :  1731
13630  :  1731
13637  :  1731
13642  :  1731
13649  :  1731
13654  :  1731
13673  :  1731
13679  :  1731
13680  :  1737
13681  :  1745
13684  :  1745
13685  :  1746
13687  :  1746
13714  :  1746
13730  :  1746
13731  :  1756
13735  :  1756
13736  :  1777
13738  :  1777
13741  :  1777
13746  :  1777
13759  :  1777
13760  :  1779
13763  :  1779
13768  :  1779
13771  :  1779
13772  :  

20531  :  2572
20548  :  2572
20559  :  2572
20570  :  2572
20576  :  2572
20578  :  2572
20583  :  2572
20594  :  2572
20599  :  2572
20611  :  2572
20617  :  2572
20625  :  2572
20632  :  2572
20648  :  2572
20654  :  2572
20659  :  2572
20661  :  2572
20675  :  2572
20678  :  2572
20703  :  2572
20708  :  2572
20709  :  2572
20721  :  2572
20730  :  2572
20738  :  2572
20745  :  2572
20754  :  2572
20762  :  2572
20765  :  2572
20774  :  2572
20781  :  2572
20782  :  2580
20797  :  2580
20799  :  2580
20801  :  2580
20806  :  2580
20812  :  2580
20820  :  2580
20833  :  2580
20835  :  2580
20838  :  2580
20843  :  2580
20844  :  2588
20856  :  2588
20857  :  2594
20859  :  2594
20868  :  2594
20876  :  2594
20880  :  2594
20885  :  2594
20901  :  2594
20911  :  2594
20921  :  2594
20922  :  2617
20928  :  2617
20950  :  2617
20953  :  2617
20966  :  2617
20971  :  2617
20978  :  2617
20983  :  2617
20985  :  2617
20987  :  2617
20994  :  2617
20998  :  2617
21000  :  2617
21009  :  

27477  :  3404
27490  :  3404
27497  :  3404
27498  :  3411
27511  :  3411
27535  :  3411
27536  :  3412
27538  :  3412
27546  :  3412
27547  :  3442
27548  :  3442
27553  :  3442
27554  :  3456
27578  :  3456
27580  :  3456
27593  :  3456
27597  :  3456
27601  :  3456
27603  :  3456
27607  :  3456
27619  :  3456
27633  :  3456
27646  :  3456
27656  :  3456
27657  :  3458
27670  :  3458
27676  :  3458
27678  :  3458
27682  :  3458
27686  :  3458
27702  :  3458
27706  :  3458
27716  :  3458
27720  :  3458
27723  :  3458
27731  :  3458
27738  :  3458
27739  :  3460
27743  :  3460
27747  :  3460
27748  :  3494
27752  :  3494
27753  :  3502
27755  :  3502
27763  :  3502
27777  :  3502
27778  :  3542
27785  :  3542
27790  :  3542
27803  :  3542
27812  :  3542
27830  :  3542
27852  :  3542
27853  :  3550
27855  :  3550
27861  :  3550
27864  :  3550
27871  :  3550
27879  :  3550
27895  :  3550
27903  :  3550
27917  :  3550
27919  :  3550
27929  :  3550
27930  :  3551
27940  :  3551
27947  :  

34759  :  4416
34767  :  4416
34777  :  4416
34786  :  4416
34788  :  4416
34800  :  4416
34809  :  4416
34812  :  4416
34813  :  4439
34820  :  4439
34843  :  4439
34854  :  4439
34864  :  4439
34866  :  4439
34868  :  4439
34883  :  4439
34884  :  4445
34894  :  4445
34907  :  4445
34950  :  4445
34959  :  4445
34972  :  4445
34977  :  4445
34989  :  4445
34996  :  4445
35015  :  4445
35020  :  4445
35021  :  4445
35023  :  4445
35029  :  4445
35032  :  4445
35035  :  4445
35036  :  4462
35038  :  4462
35044  :  4462
35048  :  4462
35051  :  4462
35062  :  4462
35066  :  4462
35067  :  4468
35084  :  4468
35087  :  4468
35091  :  4468
35095  :  4468
35096  :  4468
35102  :  4468
35127  :  4468
35146  :  4468
35154  :  4468
35159  :  4468
35165  :  4468
35167  :  4468
35177  :  4468
35190  :  4468
35201  :  4468
35212  :  4468
35213  :  4488
35220  :  4488
35221  :  4493
35224  :  4493
35232  :  4493
35236  :  4493
35246  :  4493
35248  :  4493
35250  :  4493
35271  :  4493
35273  :  

464